These are the required dependencies on Google Colab. Locally, there might be more.

In [11]:
%%capture
%pip install -U accelerate
%pip install -U peft
%pip install -U trl
%pip install -U bitsandbytes
%pip install -U transformers

Login to Huggingface to be able to save the model to your account

In [1]:
from huggingface_hub import login

hf_token = "HUGGINGFACE_ACCESS_TOKEN"
login(hf_token)

Download a quantized version of Qwen3-8B

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# model_dir = "Qwen/Qwen3-8B"
model_dir = "/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_dir, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

/home/sebastian/miniconda3/envs/finetune-qwen3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 5/5 [00:07<00:00,  1.51s/it]


In [2]:
train_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.

### Instruction:
{}

### Question:
{}

### Response:
<think>
</think>
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    messages = examples["messages"]
    system_prompt = messages[0]["content"]
    user_prompt = messages[1]["content"]
    response = messages[2]["content"]
    if not response.endswith(tokenizer.eos_token):
        response += tokenizer.eos_token
    text = train_prompt_style.format(system_prompt, user_prompt, response)
    return text, system_prompt, user_prompt, response

Load the train set used for e.g. finetuning GPT-4o and convert it to the  previous format.

In [3]:
import json
import random

data = []
with open("finetuning_files/train-new-qwen.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        data.append(json.loads(line))

data = data[0]
data = [formatting_prompts_func(x) for x in data]
raw_texts, instructions, questions, answers = zip(*data)
data = {
    "text": raw_texts,
    "system": instructions,
    "user": questions,
    "response": answers
}

data["text"][10]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nYou are driving a car and need to make a high-level driving decision.\nFirst, carefully observe the environment.\nThen, reason through your decision step by step and present it in natural language.\nFinally, return the top 3 advisable longitudinal–lateral action pairs, ranked from best to worst.\nFeasible longitudinal actions:\n  - accelerate\n  - decelerate\n  - keep\n  - stop\nFeasible lateral actions:\n  - follow_lane\n  - right\nNo past actions recorded. \n /no_think\n\n### Question:\nHere is an overview of your environment:\nThere is no left-adjacent lane. There is a right-adjacent lane with the same direction. \nYour velocity is 24.4 m/s, orientation is -0.003 rad, steering angle is 0.000 rad, and acceleration is 0.3 m/s².\nPlease adhere to the traffic regulations in Germany:\nThe maximum speed is 33.3

Create a custom dataset from the prompts

In [4]:
from datasets import Dataset

dataset = Dataset.from_dict(data)

dataset["text"][10]

'Below is an instruction that describes a task, paired with an input that provides further context.\nWrite a response that appropriately completes the request.\n\n### Instruction:\nYou are driving a car and need to make a high-level driving decision.\nFirst, carefully observe the environment.\nThen, reason through your decision step by step and present it in natural language.\nFinally, return the top 3 advisable longitudinal–lateral action pairs, ranked from best to worst.\nFeasible longitudinal actions:\n  - accelerate\n  - decelerate\n  - keep\n  - stop\nFeasible lateral actions:\n  - follow_lane\n  - right\nNo past actions recorded. \n /no_think\n\n### Question:\nHere is an overview of your environment:\nThere is no left-adjacent lane. There is a right-adjacent lane with the same direction. \nYour velocity is 24.4 m/s, orientation is -0.003 rad, steering angle is 0.000 rad, and acceleration is 0.3 m/s².\nPlease adhere to the traffic regulations in Germany:\nThe maximum speed is 33.3

In [5]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

Tryout the dataset on the base model. (Might take some time, so you can also skip it)

In [7]:
inference_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
{}

### Question:
{}

### Response:
<think>

"""
instruction = dataset[10]['system']
question = dataset[10]['user']
inputs = tokenizer(
    [inference_prompt_style.format(instruction, question) + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Response:")[1])


<think>



Okay, let's break down the problem step by step. The user is driving a car and needs to make a high-level decision based on the given environment. The main points to consider are the surroundings, current speed, and the regulations.

First, I need to check the environment. The left-adjacent lane is empty, and there's a right-adjacent lane with the same direction. The car 11842 is on the same lane and 72.5 meters in front, moving at 24.9 m/s. The truck 11844 is on the right-adjacent lane, 29.3 meters in front, moving at 24.4 m/s. There are other cars in front and behind, but none are moving near the current one. The maximum speed is 33.3 m/s, so the car 11842 is slightly ahead, but the time to collision is infinite, meaning it's not close. The car 11846 is moving faster than 25.4 m/s, which is over the limit, so that's a problem.

Now, the task is to determine the best actions. The user needs to make a high-level driving decision. The possible actions are accelerate, deceler

Initialize a LoRA version of the base model

In [6]:
from peft import LoraConfig, get_peft_model

# LoRA config
peft_config = LoraConfig(
    lora_alpha=16,                           # Scaling factor for LoRA
    lora_dropout=0.05,                       # Add slight dropout for regularization
    r=64,                                    # Rank of the LoRA update matrices
    bias="none",                             # No bias reparameterization
    task_type="CAUSAL_LM",                   # Task type: Causal Language Modeling
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],  # Target modules for LoRA
)

model = get_peft_model(model, peft_config)

Initialize the SFT trainer. You can choose the batchsize according to your hardware.

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments


# Training Arguments
training_arguments = TrainingArguments(
    output_dir="output",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=1,
    logging_steps=0.2,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="none"
)

# Initialize the Trainer
trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset,
    peft_config=peft_config,
    data_collator=data_collator,
)

Truncating train dataset: 100%|██████████| 2000/2000 [00:00<00:00, 699925.57 examples/s]
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [8]:
import gc, torch
gc.collect()
torch.cuda.empty_cache()
model.config.use_cache = False
trainer.train()

TrainOutput(global_step=1000, training_loss=0.16933707427978514, metrics={'train_runtime': 619.8072, 'train_samples_per_second': 3.227, 'train_steps_per_second': 1.613, 'total_flos': 6.746183665634304e+16, 'train_loss': 0.16933707427978514})

Test the finetuned model.

In [9]:
inference_prompt_style = """Below is an instruction that describes a task, paired with an input that provides further context.
Write a response that appropriately completes the request.
Before answering, think carefully about the question and create a step-by-step chain of thoughts to ensure a logical and accurate response.

### Instruction:
{}

### Question:
{}

### Response:
<think>

"""

instruction = dataset[10]['system']
question = dataset[10]['user']
inputs = tokenizer(
    [inference_prompt_style.format(instruction, question) + tokenizer.eos_token],
    return_tensors="pt"
).to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    eos_token_id=tokenizer.eos_token_id,
    use_cache=True,
)
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(response[0].split("### Response:")[1])


<think>


</think>
{
  "best_combination": {
    "lateral_action": "follow_lane",
    "longitudinal_action": "accelerate"
  }
  ,
  "second_best_combination": {
    "lateral_action": "right",
    "longitudinal_action": "accelerate"
  }
  ,
  "third_best_combination": {
    "lateral_action": "follow_lane",
    "longitudinal_action": "accelerate"
  }
}


Push the finetuned model to HuggingFace

In [10]:
local_folder_path = "/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64"

# Save model and tokenizer locally
model.save_pretrained(local_folder_path)
tokenizer.save_pretrained(local_folder_path)

('/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64/tokenizer_config.json',
 '/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64/special_tokens_map.json',
 '/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64/chat_template.jinja',
 '/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64/vocab.json',
 '/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64/merges.txt',
 '/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64/added_tokens.json',
 '/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64/tokenizer.json')

In [11]:
# new_model_name = "Qwen-3-8B-HighD"
# model.push_to_hub(new_model_name)
# tokenizer.push_to_hub(new_model_name)

Download the actual model and the LoRa Adapters like this:

In [ ]:
%git lfs clone https://huggingface.co/Qwen/Qwen-3-8B

%git lfs clone https://huggingface.co/Bastilling/Qwen-3-8B-HighD-2000

Then merge the two by running this:

In [11]:
import os

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

base_model_path = "/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B"
adapter_path = "/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-Adapter-64"

# Load tokenizer (optional)
tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=True)
save_path = "/home/sebastian/Documents/Uni/LlamaCPP/Qwen3-8B-highD-64"
tokenizer.save_pretrained(save_path)

# Load full-precision base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    device_map="auto",
    torch_dtype=torch.float16,  # or torch.float32
    trust_remote_code=True,
)

# Load and merge adapter
model = PeftModel.from_pretrained(base_model, adapter_path)
model = model.merge_and_unload()

# Save merged full model
model.save_pretrained(save_path)

Loading checkpoint shards: 100%|██████████| 5/5 [00:01<00:00,  3.95it/s]


Then clone the llama.cpp repo and build the code. Afterwards run the conversion script:

In [ ]:
%python convert_hf_to_gguf.py /home/sebastian/Documents/Uni/LlamaCPP/qwen3-8b-merged-2000 --outfile /home/sebastian/Documents/Uni/LlamaCPP/qwen3-8b-highD-2000-gguf/qwen3-8b-highD-2000-f16.gguf --outtype f16

Next, create a model file inside the gguf repo. It should be called "Modelfile" and contain:
```
FROM ./Qwen3-8B-highD-64-f16.gguf

TEMPLATE """<|im_start|>system
{{ .System }}<|im_end|>
<|im_start|>user
{{ .Prompt }}<|im_end|>
<|im_start|>assistant
"""

PARAMETER stop "<|im_start|>"
PARAMETER stop "<|im_end|>"
PARAMETER num_ctx 4096
```

Next, start ollama server and add the model (I didn't test the quantizing so far, but it should work in theory. Otherwise, just remove the last argument.):

In [ ]:
%ollama serve
%ollama create qwen3-8b-highD -f Modelfile --quantize q5_K_M